In [1]:
%load_ext autoreload
%autoreload 2

import math
import argparse
import h5py
import importlib
import numpy as np
import torch
import time
import sys
from torch.utils.data import DataLoader
from torch.nn.utils import clip_grad_norm_

from models.vae_flow import *
from models.flow import add_spectral_norm, spectral_norm_power_iteration
from configs import Configs
from utils.plotting import get_plots, get_features
from utils.detector_map import get_projections, create_map, layer_bottom_pos
from tqdm import tqdm

import k_diffusion as K

import utils.metrics as metrics
import utils.plotting as plotting


cfg = Configs()

print(cfg.__dict__)

/beegfs/desy/user/buhmae/conda/envs/torch_113/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 30/30 [00:00<00:00, 287.92it/s]


{'name': 'CD_', 'comet_project': 'calo-consistency', 'Acomment': 'long baseline with lat_dim = 0, max_iter 1M, lr=1e-4 fixed, num_steps=18, bs=256, simga_max=80, epoch=2M, EMA', 'log_comet': True, 'model_name': 'epicVAE_nFlow_kDiffusion', 'latent_dim': 0, 'beta_1': 0.0001, 'beta_T': 0.02, 'sched_mode': 'quardatic', 'flexibility': 0.0, 'truncate_std': 2.0, 'latent_flow_depth': 14, 'latent_flow_hidden_dim': 256, 'num_samples': 4, 'features': 4, 'sample_num_points': 2048, 'kl_weight': 0.001, 'residual': False, 'cond_features': 2, 'norm_cond': True, 'kld_min': 1.0, 'use_epic': False, 'epic_layers': 5, 'hidden_dim': 128, 'sum_scale': 0.001, 'weight_norm': True, 'flow_model': 'PiecewiseRationalQuadraticCouplingTransform', 'flow_transforms': 10, 'flow_layers': 2, 'flow_hidden_dims': 128, 'tails': 'linear', 'tail_bound': 10, 'dataset': 'x36_grid', 'dataset_path': '/beegfs/desy/user/akorol/data/calo-clouds/hdf5/high_granular_grid/train/10-90GeV_x36_grid_regular_524k_float32.hdf5', 'quantized_po

In [2]:
importlib.reload(plotting)

100%|██████████| 30/30 [00:00<00:00, 285.34it/s]


<module 'utils.plotting' from '/home/buhmae/6_PointCloudDiffusion/utils/plotting.py'>

In [3]:
# path = '/beegfs/desy/user/akorol/data/calo-clouds/hdf5/high_granular_grid/validation/10-90GeV_x36_grid_regular.hdf5'
# path = '/beegfs/desy/user/akorol/data/calo-clouds/hdf5/high_granular_grid/validation/50GeV_x36_grid_regular_2k_Z4.hdf5'
path = '/beegfs/desy/user/akorol/data/calo-clouds/hdf5/all_steps/validation/photon-showers_10-90GeV_A90_Zpos4.slcio.hdf5'
real_showers = h5py.File(path, 'r')['events'][:]
real_showers[:, -1] = real_showers[:, -1] * 1000   # GeV to MeV
print(real_showers.shape)

(40000, 4, 40000)


In [4]:
MAP = create_map()
real_events = get_projections(real_showers[:], MAP, layer_bottom_pos)
len(real_events)
len(real_events[0])

100%|██████████| 40000/40000 [03:05<00:00, 215.44it/s]


30

In [5]:
importlib.reload(plotting)

st = time.time()
dict = plotting.get_features(real_events[:])
print(time.time()-st)

100%|██████████| 40000/40000 [01:43<00:00, 386.75it/s]


115.44061851501465


In [6]:
print(dict.keys())

print(dict['binned_layer_e'].shape)

dict_keys(['e_radial', 'e_sum', 'hits', 'occ', 'e_layers_distibution', 'e_layers', 'occ_layers', 'e_radial_lists', 'hits_noThreshold', 'binned_layer_e', 'binned_radial_e'])
(10, 40000)


In [8]:
dict['e_radial_lists'][3].shape

(2, 594)

In [9]:
dict['occ'][3]

594

In [10]:
dict['occ_layers'].shape

(40000, 30)

In [11]:
dict['e_layers_distibution'].shape

(40000, 30)

# caluclate percentile edges for layer wise occupancy

In [12]:
importlib.reload(metrics)

percentile_edges_layer_occupancy = metrics.percentile_edges_layer_occupancy(dict['occ_layers'])

print(percentile_edges_layer_occupancy)
print(len(percentile_edges_layer_occupancy))

[0, 8, 11, 13, 15, 16, 18, 19, 21, 24, 29]
11


# binned occupancy

In [13]:
occ_layers = dict['occ_layers']

binned_layer_occ = metrics.binned_layer_occupcany(occ_layers, bin_edges = percentile_edges_layer_occupancy)

print(binned_layer_occ.shape)

(10, 40000)


# binned e layers

In [14]:
e_layers_distibution = dict['e_layers_distibution']

binned_layer_e = metrics.binned_layer_energy(e_layers_distibution, bin_edges = percentile_edges_layer_occupancy)

print(binned_layer_e.shape)

(10, 40000)


# percentile edges for raidal occpuancy

In [15]:
occ_radial = dict['e_radial_lists'][0]

importlib.reload(metrics)
percentile_edges_radial_occupancy = metrics.percentile_edges_radial_occupancy(occ_radial)

print(percentile_edges_radial_occupancy)

[0, 6.558, 9.849, 12.96, 17.028, 23.434, 33.609, 40.119, 48.491, 68.808, 300]


# binned occ radial (for testing)

In [16]:
importlib.reload(metrics)

binned_radial_occ_sum = metrics.binned_radial_occupancy_sum(occ_radial, bin_edges = percentile_edges_radial_occupancy)

print(binned_radial_occ_sum.shape)
print(binned_radial_occ_sum / binned_radial_occ_sum.sum())

(10,)
[0.10018753 0.10017056 0.09973458 0.10220211 0.10028216 0.10016354
 0.0999101  0.10001376 0.10444626 0.0928894 ]


# binned e radial distributions

In [17]:
importlib.reload(metrics)

binned_radial_e = metrics.binned_radial_energy(dict['e_radial_lists'], bin_edges = percentile_edges_radial_occupancy)

print(binned_radial_e.shape)

(10, 40000)
